## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [3]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [118]:
def gini(p):
    return


def entropy(p):
    return


def misclass(p):   
    return

In [123]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self,
                 min_samples_split=2,
                 max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            raise ValueError('invalid criterion name')

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            raise ValueError('invalid max_features name')

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return # Ваш код
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return # Ваш код 

    def __misclass(self, l_c, l_s, r_c, r_s):
        return # Ваш код

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        max_features = max(1, int(np.sqrt(n_feature)))
        return feature_ids[:max_features]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        max_features = max(1, int(np.log2(n_feature)))
        return feature_ids[:max_features]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Сортируем значения свойства по возрастанию и приводим к аналогичной последовательности целевые переменные
        sorted_x, sorted_y = self.__sort_samples(x, y)
        print(sorted_x)
        print(sorted_y)
        # Количество уникальных вариантов целевых переменной
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        print(splitted_sorted_y)
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        print(np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0])
        print(r_border_ids)
        print(splitted_sorted_y[69:])

        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
    
    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        n_samples, n_features = x.shape
        for feature_id in self.get_feature_ids(n_features):
            threshold = self.__find_threshold(x[:,feature_id], y)
            break
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [112]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df = df.sample(n=100)
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
27160,0,0.007831,51,0,0.764725,3123.0,8,0,1,0,2.0
50257,0,1.098951,36,0,0.216481,8166.0,3,0,1,0,1.0
7506,0,0.001384,35,0,0.094758,7935.0,5,0,0,0,3.0
128162,0,0.801014,38,1,0.066206,7083.0,5,0,0,0,0.0
101729,0,0.348742,56,0,0.363611,7100.0,10,0,3,0,1.0


In [113]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])
print(x[0][6], x[0][8], x[0][0], x[0][3], x[0][2], x[0][9], x[0][7], x[0][5], x[0][1], x[0][4])
print(x[1][2], x[1][6], x[1][7], x[1][8], x[1][3], x[1][0], x[1][9], x[1][5], x[1][1], x[1][4])
x.shape

(0.0, 0.0, 0.0078305939999999997, 0.76472471200000003, 0.0, 2.0, 1.0, 8.0, 51.0, 3123.0)
(0.0, 0.0, 1.0, 0.0, 0.21648096, 1.098950525, 1.0, 3.0, 36.0, 8166.0)


(100, 10)

In [124]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
my_clf.fit(x, y)

[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   2.69919000e-04
   1.38360400e-03   4.75981000e-03   6.24980500e-03   6.34439000e-03
   7.44111400e-03   7.71235900e-03   7.83059400e-03   8.35395100e-03
   9.43617700e-03   1.08513500e-02   1.31380070e-02   1.62104370e-02
   1.82028410e-02   2.02554510e-02   2.09339770e-02   2.41068840e-02
   3.24569810e-02   3.28454920e-02   3.42528810e-02   3.43085450e-02
   4.15625650e-02   4.28805930e-02   4.61747730e-02   4.91358060e-02
   5.66572240e-02   5.70377180e-02   5.85387990e-02   6.41558620e-02
   6.58416510e-02   6.85272820e-02   6.91827480e-02   7.09263730e-02
   7.13774270e-02   7.29773670e-02   7.61910910e-02   8.42880520e-02
   8.83190880e-02   9.78484650e-02   1.02940537e-01   1.05145758e-01
   1.05289942e-01   1.12433173e-01   1.30259735e-01   1.32628089e-01
   1.35371075e-01   1.49965359e-01   1.64564957e-01   1.93887032e-01
   1.97200676e-01   1.98869140e-01

TypeError: 'NoneType' object has no attribute '__getitem__'

In [7]:
np.unique(y)

array([0, 1])

In [45]:
d = np.array([3, 3, 1, 2, 1, 2, 3, 25, 4])
d_y = np.bincount(d)
print(d)
d_y

[ 3  3  1  2  1  2  3 25  4]


array([0, 2, 2, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1])

In [21]:
a = np.array([6, 8, 2, 7, 1, 10, 6, 8, 2]).reshape(3, 3)
a

array([[ 6,  8,  2],
       [ 7,  1, 10],
       [ 6,  8,  2]])

In [31]:
a[:,1]

array([8, 1, 8])

In [32]:
a_idx = a[:,1].argsort()
a_idx

array([1, 0, 2])

In [33]:
a[a_idx]

array([[ 7,  1, 10],
       [ 6,  8,  2],
       [ 6,  8,  2]])

In [10]:
a[1:]

array([[ 7,  1,  4, 10],
       [ 6,  8,  2,  1]])

In [11]:
a[:-1]

array([[ 6,  8,  2,  1],
       [ 7,  1,  4, 10]])

In [12]:
np.where(a[:-1] != a[1:])

(array([0, 0, 0, 0, 1, 1, 1, 1]), array([0, 1, 2, 3, 0, 1, 2, 3]))

In [16]:
np.log2(4)
c = np.array([6, 8, 2, 1, 7, 1, 4, 10, 6, 8, 2, 1])

array([6])

In [29]:
x.argsort()[1]

array([2, 6, 7, 8, 3, 0, 9, 5, 1, 4])

In [46]:
np.unique(y).size

2

In [64]:
def __sort_samples(x, y):
    sorted_idx = x.argsort()
    return x[sorted_idx], y[sorted_idx]

In [65]:
sorted_x, sorted_y = __sort_samples(x, y)

In [93]:
splitted_sorted_y = sorted_y[2:-2]
r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (2 + 1)
splitted_sorted_y[1:]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
my_clf.fit(x, y)

In [5]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [6]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.00365805625916
1.13050794601


## Проверка качества работы

In [7]:
gkf = KFold(n_splits=5, shuffle=True)

In [8]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

1.0
1.0
0.999958426873
1.0
0.999958425144


In [9]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.894321110834
0.895859316538
0.891078406918
0.890828968155
0.891406477362


# Применить для задачи Titanic 